In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, BatchNorm, global_add_pool, ChebConv, global_max_pool, SAGPooling, GATConv, GATv2Conv, TransformerConv, SuperGATConv, global_mean_pool, Linear
from torch.nn import BatchNorm1d
from math import floor
import torch
import torch_geometric
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from pathlib import Path

In [3]:
class EEGConvNetMiniV3(nn.Module):
    """Same as EEGGraphConvNet but with fewer 
    convolutional layers
    """
    def __init__(self, input_size, **kwargs):
        super(EEGConvNetMiniV3, self).__init__()
        self.input_size = input_size
        # Layers definition
        # Graph convolutional layers
        self.conv1 = GCNConv(input_size, 32, cached=True, normalize=False)
        self.conv2 = GCNConv(32, 64, cached=True, normalize=False)
        
        # Batch normalization
        self.batch_norm1 = BatchNorm(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm2 = BatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

        
        # Fully connected layers
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 2)
        
        # Xavier initializacion for fully connected layers
        self.fc1.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc2.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc3.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        
        
    def forward(self, x, edge_index, edge_weigth, batch):
        x = F.leaky_relu(self.batch_norm1(self.conv1(x, edge_index, edge_weigth)), negative_slope=0.01)
        x = F.leaky_relu(self.batch_norm2(self.conv2(x, edge_index, edge_weigth)), negative_slope=0.01)
        mean_pool = global_add_pool(x, batch=batch)
        out = F.leaky_relu(self.fc1(mean_pool), negative_slope=0.01)        
        out = F.leaky_relu(self.fc2(out), negative_slope=0.01)
        out = F.leaky_relu(self.fc3(out))
        return out
    


In [4]:
def create_corrected_data_list(path):
    data_list = list()
    for file in path.iterdir():
        data_list.append(torch.load(file))
    corrected_data_list = list()
    for data in data_list:
    # print(data)
        data = torch_geometric.data.Data(
            x=torch.tensor(data.x),
            edge_index=torch.tensor(data.edge_index),
            edge_attr=torch.tensor(data.edge_attr),
            label=torch.tensor(data.label),
        )
        corrected_data_list.append(data)
        
    rm = [
      7,
      14+1,
      14+2,
      17+3,
      17+4,
      26+5,
      38+6,
      54+7,
      65+8,
      69+9
      ]

    dl = list()
    start = 0
    for r in rm:
        dl.extend(corrected_data_list[start:r])
        start = r + 1

    dl.extend(corrected_data_list[start:])
    dl_filterd = list()
    for data in dl:
        if data.label == 2:
            # print(data.label)
            # if data.label == 2:
            data.label = torch.tensor(1)
        dl_filterd.append(data)

    len(dl_filterd)
    return dl_filterd
            
def print_classification_report(y_pred, y_true):
    print(classification_report(y_pred, y_true))
    

## Moments Pearson

In [5]:
path = Path('graphs/moments_pearson/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_55772\660667472.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [7]:
model = EEGConvNetMiniV3(input_size=6)

In [8]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 5170


EEGConvNetMiniV3(
  (conv1): GCNConv(6, 32)
  (conv2): GCNConv(32, 64)
  (batch_norm1): BatchNorm(32)
  (batch_norm2): BatchNorm(64)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
)

In [9]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=0.714760	 acc=0.641026
Test  : epoch=1	 loss=6.164265	 acc=0.500000

Train : epoch=2	 loss=0.749627	 acc=0.653846
Test  : epoch=2	 loss=21.235391	 acc=0.500000

Train : epoch=3	 loss=4.073162	 acc=0.653846
Test  : epoch=3	 loss=48.044988	 acc=0.500000

Train : epoch=4	 loss=4.463126	 acc=0.653846
Test  : epoch=4	 loss=88.120697	 acc=0.500000

Train : epoch=5	 loss=4.614084	 acc=0.653846
Test  : epoch=5	 loss=135.984260	 acc=0.500000

Train : epoch=6	 loss=4.411246	 acc=0.653846
Test  : epoch=6	 loss=144.436378	 acc=0.500000

Train : epoch=7	 loss=2.849206	 acc=0.653846
Test  : epoch=7	 loss=141.764202	 acc=0.500000

Train : epoch=8	 loss=2.895817	 acc=0.653846
Test  : epoch=8	 loss=153.311225	 acc=0.500000

Train : epoch=9	 loss=2.943150	 acc=0.653846
Test  : epoch=9	 loss=159.682459	 acc=0.500000

Train : epoch=10	 loss=2.898514	 acc=0.653846
Test  : epoch=10	 loss=160.611525	 acc=0.500000



In [10]:
print_classification_report(y_pred, y_true)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.50      0.67        16

    accuracy                           0.50        16
   macro avg       0.50      0.25      0.33        16
weighted avg       1.00      0.50      0.67        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

## Moments PLI

In [11]:
path = Path('graphs/moments_pli/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_55772\660667472.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [12]:
model = EEGConvNetMiniV3(input_size=6)

In [13]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 5170


EEGConvNetMiniV3(
  (conv1): GCNConv(6, 32)
  (conv2): GCNConv(32, 64)
  (batch_norm1): BatchNorm(32)
  (batch_norm2): BatchNorm(64)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
)

In [14]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=0.711133	 acc=0.653846
Test  : epoch=1	 loss=311.892502	 acc=0.500000

Train : epoch=2	 loss=0.747816	 acc=0.653846
Test  : epoch=2	 loss=698.035482	 acc=0.500000

Train : epoch=3	 loss=3.124259	 acc=0.653846
Test  : epoch=3	 loss=1562.786970	 acc=0.500000

Train : epoch=4	 loss=4.326050	 acc=0.653846
Test  : epoch=4	 loss=2773.441826	 acc=0.500000

Train : epoch=5	 loss=4.588888	 acc=0.653846
Test  : epoch=5	 loss=4280.791668	 acc=0.500000

Train : epoch=6	 loss=4.702327	 acc=0.653846
Test  : epoch=6	 loss=5058.989364	 acc=0.500000

Train : epoch=7	 loss=2.760034	 acc=0.653846
Test  : epoch=7	 loss=4774.280800	 acc=0.500000

Train : epoch=8	 loss=2.746957	 acc=0.653846
Test  : epoch=8	 loss=4371.992555	 acc=0.500000

Train : epoch=9	 loss=2.753273	 acc=0.653846
Test  : epoch=9	 loss=4243.688229	 acc=0.500000

Train : epoch=10	 loss=2.754408	 acc=0.653846
Test  : epoch=10	 loss=4151.422110	 acc=0.500000



In [15]:
print_classification_report(y_pred, y_true)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.50      0.67        16

    accuracy                           0.50        16
   macro avg       0.50      0.25      0.33        16
weighted avg       1.00      0.50      0.67        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

## Raw Pearson

In [16]:
path = Path('graphs/raw_pearson/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_55772\660667472.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x=torch.tensor(data.x),
C:\Users\mashh\AppData\Local\Temp\ipykernel_55772\660667472.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [17]:
model = EEGConvNetMiniV3(input_size=153550)

In [18]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 4918578


EEGConvNetMiniV3(
  (conv1): GCNConv(153550, 32)
  (conv2): GCNConv(32, 64)
  (batch_norm1): BatchNorm(32)
  (batch_norm2): BatchNorm(64)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
)

In [19]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=3.217563	 acc=0.615385
Test  : epoch=1	 loss=3.375572	 acc=0.500000

Train : epoch=2	 loss=3.674591	 acc=0.653846
Test  : epoch=2	 loss=23.631988	 acc=0.500000

Train : epoch=3	 loss=3.798907	 acc=0.653846
Test  : epoch=3	 loss=28.706666	 acc=0.500000

Train : epoch=4	 loss=3.679223	 acc=0.653846
Test  : epoch=4	 loss=22.760697	 acc=0.500000

Train : epoch=5	 loss=3.461848	 acc=0.653846
Test  : epoch=5	 loss=27.565115	 acc=0.500000

Train : epoch=6	 loss=3.613835	 acc=0.653846
Test  : epoch=6	 loss=18.196374	 acc=0.500000

Train : epoch=7	 loss=3.439838	 acc=0.653846
Test  : epoch=7	 loss=27.012719	 acc=0.500000

Train : epoch=8	 loss=3.553264	 acc=0.653846
Test  : epoch=8	 loss=24.826859	 acc=0.437500

Train : epoch=9	 loss=3.522751	 acc=0.653846
Test  : epoch=9	 loss=29.804806	 acc=0.500000

Train : epoch=10	 loss=3.443384	 acc=0.653846
Test  : epoch=10	 loss=29.739285	 acc=0.500000



In [20]:
print_classification_report(y_pred, y_true)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.50      0.67        16

    accuracy                           0.50        16
   macro avg       0.50      0.25      0.33        16
weighted avg       1.00      0.50      0.67        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

## Raw PLI

In [21]:
path = Path('graphs/raw_pli/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_55772\660667472.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x=torch.tensor(data.x),
C:\Users\mashh\AppData\Local\Temp\ipykernel_55772\660667472.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [22]:
model = EEGConvNetMiniV3(input_size=153550)

In [23]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 4918578


EEGConvNetMiniV3(
  (conv1): GCNConv(153550, 32)
  (conv2): GCNConv(32, 64)
  (batch_norm1): BatchNorm(32)
  (batch_norm2): BatchNorm(64)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
)

In [24]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=0.704357	 acc=0.576923
Test  : epoch=1	 loss=21.070249	 acc=0.500000

Train : epoch=2	 loss=0.694172	 acc=0.653846
Test  : epoch=2	 loss=63.116934	 acc=0.500000

Train : epoch=3	 loss=0.727171	 acc=0.653846
Test  : epoch=3	 loss=132.654999	 acc=0.500000

Train : epoch=4	 loss=0.864034	 acc=0.653846
Test  : epoch=4	 loss=266.260623	 acc=0.500000

Train : epoch=5	 loss=1.190268	 acc=0.653846
Test  : epoch=5	 loss=439.137538	 acc=0.500000

Train : epoch=6	 loss=1.765170	 acc=0.653846
Test  : epoch=6	 loss=736.246391	 acc=0.500000

Train : epoch=7	 loss=2.319989	 acc=0.653846
Test  : epoch=7	 loss=894.685017	 acc=0.500000

Train : epoch=8	 loss=2.571048	 acc=0.653846
Test  : epoch=8	 loss=922.174288	 acc=0.500000

Train : epoch=9	 loss=2.488850	 acc=0.653846
Test  : epoch=9	 loss=875.721853	 acc=0.500000

Train : epoch=10	 loss=2.520009	 acc=0.653846
Test  : epoch=10	 loss=934.102242	 acc=0.500000



In [25]:
print_classification_report(y_pred, y_true)




              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.50      0.67        16

    accuracy                           0.50        16
   macro avg       0.50      0.25      0.33        16
weighted avg       1.00      0.50      0.67        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

## PSD CSD

In [26]:
path = Path('graphs/psd_csd/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_55772\660667472.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [27]:
model = EEGConvNetMiniV3(input_size=6)

In [28]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 5170


EEGConvNetMiniV3(
  (conv1): GCNConv(6, 32)
  (conv2): GCNConv(32, 64)
  (batch_norm1): BatchNorm(32)
  (batch_norm2): BatchNorm(64)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
)

In [29]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=0.784474	 acc=0.653846
Test  : epoch=1	 loss=0.720767	 acc=0.500000

Train : epoch=2	 loss=1.204476	 acc=0.653846
Test  : epoch=2	 loss=1.165880	 acc=0.500000

Train : epoch=3	 loss=2.502766	 acc=0.653846
Test  : epoch=3	 loss=2.873973	 acc=0.500000

Train : epoch=4	 loss=3.352124	 acc=0.653846
Test  : epoch=4	 loss=3.640285	 acc=0.500000

Train : epoch=5	 loss=3.347372	 acc=0.653846
Test  : epoch=5	 loss=3.615864	 acc=0.500000

Train : epoch=6	 loss=3.343063	 acc=0.653846
Test  : epoch=6	 loss=3.624997	 acc=0.500000

Train : epoch=7	 loss=3.338414	 acc=0.653846
Test  : epoch=7	 loss=3.628245	 acc=0.500000

Train : epoch=8	 loss=3.335808	 acc=0.653846
Test  : epoch=8	 loss=3.637362	 acc=0.500000

Train : epoch=9	 loss=3.332089	 acc=0.653846
Test  : epoch=9	 loss=3.642739	 acc=0.500000

Train : epoch=10	 loss=3.328108	 acc=0.653846
Test  : epoch=10	 loss=3.648287	 acc=0.500000



In [30]:
print_classification_report(y_pred, y_true)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.50      0.67        16

    accuracy                           0.50        16
   macro avg       0.50      0.25      0.33        16
weighted avg       1.00      0.50      0.67        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

## PSD Pearson

In [31]:
path = Path('graphs/psd_pearson/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_55772\660667472.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [32]:
model = EEGConvNetMiniV3(input_size=6)

In [33]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 5170


EEGConvNetMiniV3(
  (conv1): GCNConv(6, 32)
  (conv2): GCNConv(32, 64)
  (batch_norm1): BatchNorm(32)
  (batch_norm2): BatchNorm(64)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
)

In [34]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=3.666988	 acc=0.653846
Test  : epoch=1	 loss=57.672626	 acc=0.562500

Train : epoch=2	 loss=3.786843	 acc=0.653846
Test  : epoch=2	 loss=66.697564	 acc=0.562500

Train : epoch=3	 loss=3.591644	 acc=0.653846
Test  : epoch=3	 loss=69.500766	 acc=0.500000

Train : epoch=4	 loss=3.516229	 acc=0.653846
Test  : epoch=4	 loss=60.076523	 acc=0.562500

Train : epoch=5	 loss=3.547072	 acc=0.653846
Test  : epoch=5	 loss=60.425961	 acc=0.562500

Train : epoch=6	 loss=3.490059	 acc=0.653846
Test  : epoch=6	 loss=57.565302	 acc=0.562500

Train : epoch=7	 loss=3.458268	 acc=0.653846
Test  : epoch=7	 loss=55.679485	 acc=0.562500

Train : epoch=8	 loss=3.022602	 acc=0.653846
Test  : epoch=8	 loss=27.157430	 acc=0.500000

Train : epoch=9	 loss=3.355002	 acc=0.653846
Test  : epoch=9	 loss=113.336100	 acc=0.500000

Train : epoch=10	 loss=3.803683	 acc=0.653846
Test  : epoch=10	 loss=81.941586	 acc=0.500000



In [35]:
print_classification_report(y_pred, y_true)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.50      0.67        16

    accuracy                           0.50        16
   macro avg       0.50      0.25      0.33        16
weighted avg       1.00      0.50      0.67        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le